In [1]:
import pandas as pd
import re

profile_df = pd.read_csv('../static/data/original/neighbourhood-profiles-2016-csv.csv')
profile_df.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [2]:
crime_df = pd.read_csv('../static/data/original/neighbourhood-crime-rates.csv')
crime_df.head()



,_id,OBJECTID,Neighbourhood,Hood_ID,F2020_Population_Projection,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,...,Shootings_2019,Shootings_2020,Shooting_Rate2014,Shootings_Rate2015,Shootings_Rate2016,Shootings_Rate2017,Shootings_Rate2018,Shootings_Rate2019,Shootings_Rate2020,geometry
0,1,1,Yonge-St.Clair,97,14083,16,25,34,25,28,...,0,0,0.000000,0.000000,7.722008,0.000000,0.000000,0.000000,0.000000,"{u'type': u'Polygon', u'coordinates': (((-79.3..."
1,2,2,York University Heights,27,30277,273,298,363,351,362,...,6,12,6.963789,0.000000,13.986990,34.504170,27.189610,20.102520,39.634040,"{u'type': u'Polygon', u'coordinates': (((-79.5..."
2,3,3,Lansing-Westgate,38,18146,42,81,67,84,68,...,2,1,0.000000,0.000000,5.980146,0.000000,5.737564,11.245430,5.510856,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
3,4,4,Yorkdale-Glen Park,31,17560,106,137,175,163,178,...,17,14,19.740740,19.752440,19.700550,44.312210,61.042610,100.170900,79.726650,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
4,5,5,Stonegate-Queensway,16,27410,91,74,78,98,86,...,1,0,11.702290,7.788465,0.000000,3.821754,3.759257,3.702744,0.000000,"{u'type': u'Polygon', u'coordinates': (((-79.4..."


## Fix neighbourhood names

In [7]:
# get the intersection of two data above
# Get neighbourhood names from profile_df (excluding non-neighbourhood columns)
profile_neighbourhoods = set(profile_df.columns[6:])  # Skip first 6 columns which are metadata

# Get neighbourhood names from crime_df
crime_neighbourhoods = set(crime_df["Neighbourhood"])

# Get the intersection
common_neighbourhoods = profile_neighbourhoods.intersection(crime_neighbourhoods)

# Print results
print(f"Profile neighbourhoods: {len(profile_neighbourhoods)}")
print(f"Crime neighbourhoods: {len(crime_neighbourhoods)}")
print(f"Common neighbourhoods: {len(common_neighbourhoods)}")
print("\nMissing in crime data:", profile_neighbourhoods - crime_neighbourhoods)
print("\nMissing in profile data:", crime_neighbourhoods - profile_neighbourhoods)

Profile neighbourhoods: 140
Crime neighbourhoods: 140
Common neighbourhoods: 137

Missing in crime data: {'Cabbagetown-South St. James Town', 'North St. James Town', 'Weston-Pelham Park'}

Missing in profile data: {'Cabbagetown-South St.James Town', 'Weston-Pellam Park', 'North St.James Town'}


In [8]:
# Create a mapping dictionary for the inconsistent names
name_mapping = {
    'Cabbagetown-South St.James Town': 'Cabbagetown-South St. James Town',
    'North St.James Town': 'North St. James Town',
    'Weston-Pellam Park': 'Weston-Pelham Park'
}

# Replace the names in crime_df
crime_df['Neighbourhood'] = crime_df['Neighbourhood'].replace(name_mapping)

# Verify the intersection again
crime_neighbourhoods = set(crime_df["Neighbourhood"])
common_neighbourhoods = profile_neighbourhoods.intersection(crime_neighbourhoods)
print(f"Common neighbourhoods after fix: {len(common_neighbourhoods)}")

Common neighbourhoods after fix: 140


## Clean Crime Data

In [9]:
crime_df.describe()

,_id,OBJECTID,Hood_ID,F2020_Population_Projection,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_2019,...,Shootings_2018,Shootings_2019,Shootings_2020,Shooting_Rate2014,Shootings_Rate2015,Shootings_Rate2016,Shootings_Rate2017,Shootings_Rate2018,Shootings_Rate2019,Shootings_Rate2020
count,140.0000,140.0000,140.0000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,...,140.000000,140.000000,140.00000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,70.5000,70.5000,70.5000,21728.871429,117.892857,127.500000,132.807143,134.792857,140.057143,147.292857,...,3.050000,3.514286,3.30000,6.293039,10.751440,14.489193,13.458111,14.214441,16.786570,14.426955
std,40.5586,40.5586,40.5586,11839.460093,105.241052,118.164773,124.394007,136.004840,143.163677,148.913740,...,3.929825,4.404846,4.62508,9.152261,15.338856,19.019151,15.484375,16.158190,19.271088,17.827221
min,1.0000,1.0000,1.0000,7130.000000,14.000000,11.000000,10.000000,20.000000,14.000000,16.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.7500,35.7500,35.7500,13227.250000,54.750000,56.750000,56.750000,56.000000,58.500000,66.750000,...,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.276419,0.000000
50%,70.5000,70.5000,70.5000,18378.000000,87.000000,98.000000,97.000000,96.000000,95.500000,99.500000,...,2.000000,2.000000,2.00000,3.770907,4.999674,8.428887,8.707171,8.356319,10.350560,7.677692
75%,105.2500,105.2500,105.2500,26598.250000,141.000000,152.250000,160.250000,159.750000,167.000000,175.500000,...,4.000000,5.000000,5.00000,9.064886,15.637293,18.747100,19.466133,22.442170,24.620350,21.048253
max,140.0000,140.0000,140.0000,87808.000000,708.000000,806.000000,866.000000,869.000000,897.000000,911.000000,...,24.000000,30.000000,34.00000,54.392170,84.417870,117.233300,69.444440,82.162510,100.170900,102.933600


In [25]:
# Count NaN values for each column
nan_counts = crime_df.isna().sum()

# Show only columns that have NaN values (if any)
columns_with_nans = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
print(columns_with_nans)

Columns with NaN values:
Series([], dtype: int64)


### Melting

In [31]:
# First, identify the crime types and years
# Let's separate the columns into groups
base_cols = ['_id', 'OBJECTID', 'Neighbourhood', 'Hood_ID', 'F2020_Population_Projection', 'geometry']
crime_cols = [col for col in crime_df.columns if col not in base_cols]

# Melt the dataframe
crime_df_long = pd.melt(
    crime_df,
    id_vars=base_cols,
    value_vars=crime_cols,
    var_name='crime_type_year',
    value_name='crime_rate'
)

# Split crime_type_year into crime_type and year
crime_df_long[['crime_type', 'year']] = crime_df_long['crime_type_year'].str.extract(r'(.+)_(\d{4})')

# Drop the temporary column and reorder
crime_df_long = crime_df_long.drop('crime_type_year', axis=1)
crime_df_long = crime_df_long[['year', 'Neighbourhood', 'crime_type', 'crime_rate']].rename(columns={'Neighbourhood': 'neighbourhood'})

# Display the first few rows
print("Shape of original data:", crime_df.shape)
print("Shape of long format data:", crime_df_long.shape)
print("\nFirst few rows of the long format data:")
crime_df_long.head()

Shape of original data: (140, 104)
Shape of long format data: (13720, 4)

First few rows of the long format data:


,year,neighbourhood,crime_type,crime_rate
0,2014,Yonge-St.Clair,Assault,16.0
1,2014,York University Heights,Assault,273.0
2,2014,Lansing-Westgate,Assault,42.0
3,2014,Yorkdale-Glen Park,Assault,106.0
4,2014,Stonegate-Queensway,Assault,91.0


In [58]:
# First, identify the crime types and years
base_cols = ['_id', 'OBJECTID', 'Neighbourhood', 'Hood_ID', 'F2020_Population_Projection', 'geometry']

# Separate count and rate columns
count_cols = [col for col in crime_df.columns 
              if col not in base_cols and 'Rate' not in col]
rate_cols = [col for col in crime_df.columns if 'Rate' in col]

def format_crime_type_regex(text):
    # 在大写字母前加空格
    text = re.sub(r'(?<!^)(?=[A-Z])', ' ', text)
    
    # 定义需要小写的词
    lower_case_words = {'and', 'or', 'of', 'in', 'the', 'on', 'at'}
    
    # 处理每个词
    words = text.split()
    formatted_words = [word.lower() if word.lower() in lower_case_words else word 
                      for word in words]
    
    return ' '.join(formatted_words)

# Melt count columns
count_df = pd.melt(
    crime_df,
    id_vars=base_cols,
    value_vars=count_cols,
    var_name='crime_type_year',
    value_name='crime_count'
)

# Melt rate columns
rate_df = pd.melt(
    crime_df,
    id_vars=base_cols,
    value_vars=rate_cols,
    var_name='crime_type_year',
    value_name='crime_rate'
)

# Extract crime type and year from count_df
count_df[['crime_type', 'year']] = count_df['crime_type_year'].str.extract(r'(.+)_(\d{4})')

# Extract crime type and year from rate_df (note the different pattern for rate columns)
rate_df[['crime_type', 'year']] = rate_df['crime_type_year'].str.extract(r'(.+)_Rate(\d{4})')

# Drop temporary columns
count_df = count_df.drop('crime_type_year', axis=1)
rate_df = rate_df.drop('crime_type_year', axis=1)

# Merge count and rate dataframes
crime_df_long = pd.merge(
    count_df,
    rate_df[['Neighbourhood', 'crime_type', 'year', 'crime_rate']],
    on=['Neighbourhood', 'crime_type', 'year']
)

# Select and rename final columns
crime_df_long = crime_df_long[['Neighbourhood', 'crime_type', 'year', 'crime_count', 'crime_rate']].rename(
    columns={'Neighbourhood': 'neighbourhood'}
).sort_values(by=['neighbourhood', 'crime_type', 'year'])


crime_df_long["crime_type"] = crime_df_long['crime_type'].apply(format_crime_type_regex)

# Display the results
print("Shape of original data:", crime_df.shape)
print("Shape of long format data:", crime_df_long.shape)

# Show unique crime types to verify formatting
print("\nUnique crime types after formatting:")
print(sorted(crime_df_long['crime_type'].unique()))

# Show a few example rows
print("\nSample rows:")
crime_df_long.head()

Shape of original data: (140, 104)
Shape of long format data: (6580, 5)

Unique crime types after formatting:
['Assault', 'Auto Theft', 'Break and Enter', 'Homicide', 'Robbery', 'Shootings', 'Theft Over']

Sample rows:


,neighbourhood,crime_type,year,crime_count,crime_rate
79,Agincourt North,Assault,2014,67,219.9462
219,Agincourt North,Assault,2015,77,255.2712
359,Agincourt North,Assault,2016,78,260.3906
499,Agincourt North,Assault,2017,73,240.5034
639,Agincourt North,Assault,2018,80,259.7656


In [59]:
# export crime_df_long to csv
crime_df_long.to_csv('../static/data/processed/neighbourhood-crime-rates.csv', index=False)


# Clean Profile Data

In [24]:
# Count NaN values for each column
nan_counts = profile_df.isna().sum()

# Show only columns that have NaN values (if any)
columns_with_nans = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
print(columns_with_nans)

# For columns with NaN values, show unique Category-Topic combinations
if len(columns_with_nans) > 0:
    for col in columns_with_nans.index:
        print(f"\nUnique Category-Topic combinations where {col} is NaN:")
        unique_combinations = profile_df[profile_df[col].isna()][['Category', 'Topic']].drop_duplicates()
        print(unique_combinations)

Columns with NaN values:
City of Toronto                  7
Agincourt North                 56
Agincourt South-Malvern West    56
Alderwood                       56
Annex                           56
                                ..
Wychwood                        56
Yonge-Eglinton                  56
Yonge-St.Clair                  56
York University Heights         56
Yorkdale-Glen Park              56
Length: 141, dtype: int64

Unique Category-Topic combinations where City of Toronto is NaN:
                       Category                      Topic
0     Neighbourhood Information  Neighbourhood Information
1960            Journey to work      Commuting destination

Unique Category-Topic combinations where Agincourt North is NaN:
             Category                                    Topic
946            Income            Income of individuals in 2015
1014           Income             Income of households in 2015
1076           Income      Income of economic families in 2015
168